In [1]:
# Random Forest Baseline on Fingerprints

import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit import DataStructs
from rdkit.Chem import rdMolDescriptors
from sklearn.ensemble import RandomForestRegressor

In [39]:
train_data_path = 'data/retrospective_train.pickle'
test_data_path = 'data/retrospective_test.pickle'
p450_train_data_path = 'data/p450_train.pickle'
p450_test_data_path = 'data/p450_test.pickle'

In [40]:
train = pd.read_pickle(train_data_path)
test = pd.read_pickle(test_data_path)
p450_train = pd.read_pickle(p450_train_data_path)
p450_test = pd.read_pickle(p450_test_data_path)

In [41]:
def make_labels(pickle_data, longest_molecule):
    reaction_sites = pickle_data['parent_centres']
    Y = np.zeros(longest_molecule)
   
    if len(reaction_sites) > 0:
        for site in reaction_sites:
            Y[int(site)] = 1
    return Y

def make_label_all_df(df, longest_molecule):
    Y = []
    for i in range(len(df)):
        label = make_labels(df.iloc[i], longest_molecule)
        Y.append(label)
    df['Y'] = Y
    return df

def make_nha(df):
    nhas = []
    for i in range(len(df)):
        smiles_i = df.loc[i, 'reactant']
        mol_i = Chem.MolFromSmiles(smiles_i)
        nha_i = mol_i.GetNumHeavyAtoms()
        nhas.append(nha_i)
    df['nha'] = nhas
    return df
    
#cutting the dummy atoms out of the predictions
def snip(pred, test):
    all_preds = np.empty([0])
    for i in range(len(pred)):
        nha_i = test.loc[i, 'nha']
        p_i = pred[i][0:nha_i]
        all_preds = np.concatenate((all_preds, p_i))
    return all_preds

def simplify(pred):
    sim_pred = np.zeros(len(pred))
    for i in range(len(pred)):
        if pred[i] > 0.5:
            sim_pred[i] = 1
        else:
            pass
    return sim_pred

def score(true, pred):
    true_p = np.mean(true)
    pred_p = np.mean(pred)
    if pred_p == 0:
        pred_p = 1e-10
        
    print('true_p', true_p)
    print('pred_p', pred_p)
    tp = len(np.where(true + pred == 2)[0])
    tn = len(np.where(true + pred == 0)[0])
    fp = len(np.where(true - pred == -1)[0])
    fn = len(np.where(pred - true == -1)[0])
    print("tp", tp, "tn", tn, "fp", fp, "fn", fn)
    fit = -tp * np.log(pred_p) - tn * np.log(1-pred_p) + fp * np.log(true_p) + fn * np.log(1-true_p)

    return fit

def rxn_encoding(df_row, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives, 
                 unique_p450s, include_p450s):
  
    rxn = []
    rxn += [int(df_row['reagent'] == x) for x in unique_reagents]
    rxn += [int(df_row['oxidant'] == x) for x in unique_oxidants]
    rxn += [int(df_row['solvent'] == x) for x in unique_solvents]
    rxn += [int(df_row['acid'] == x) for x in unique_acids]
    rxn += [int(df_row['additive'] == x) for x in unique_additives]
    if include_p450s == True:
        rxn += [int(df_row['P450'] == x) for x in unique_p450s]
    return rxn

def rxn_encoding_all_df(df, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives, 
                        unique_p450s, include_p450s=False):
    rxn_encodings = []
    for i in range(len(df)):
        encoding_i = rxn_encoding(df.iloc[i], unique_reagents, unique_oxidants, unique_solvents, unique_acids, 
                                  unique_additives, unique_p450s, include_p450s)
        rxn_encodings.append(encoding_i)
    df['r'] = rxn_encodings
    return df

def make_fps(df):
    mols = [Chem.MolFromSmiles(s) for s in df['reactant']]
    df['mol'] = mols
    fps = [rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, 2, 2048) for mol in df['mol']]
    df['fps'] = fps
    return df

def rf_input(df):
    inputs = []
    for i in range(len(df)):
        fps_i = df.loc[i, 'fps']
        r_i = df.loc[i, 'r']
        input_i = np.concatenate((fps_i, r_i))
        inputs.append(input_i)
    df['rf_input'] = inputs
    return df

In [42]:
unique_reagents = train['reagent'].unique().tolist()
unique_oxidants = train['oxidant'].unique().tolist()
unique_solvents = train['solvent'].unique().tolist()
unique_acids = train['acid'].unique().tolist()
unique_additives = train['additive'].unique().tolist()
unique_p450s = p450_train['P450'].unique().tolist()

In [43]:
l = 83
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train = make_label_all_df(train, l)
train = make_nha(train)
train = rxn_encoding_all_df(train, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives,
                            unique_p450s, include_p450s=False)
train = make_fps(train)
train = rf_input(train)


test = make_label_all_df(test, l)
test = make_nha(test)
test = rxn_encoding_all_df(test, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives,
                            unique_p450s, include_p450s=False)
test = make_fps(test)
test = rf_input(test)

In [44]:
rf = RandomForestRegressor()
rf.fit(train['rf_input'].tolist(), train['Y'].tolist())

RandomForestRegressor()

In [45]:
preds = rf.predict(test['rf_input'].tolist())
p = snip(preds, test)
true = snip(test['Y'].tolist(), test)
p = simplify(p)
rf_score = score(true, p)
rf_score

true_p 0.05740740740740741
pred_p 0.005555555555555556
tp 0 tn 506 fp 3 fn 31


-7.586551823436824

In [46]:
## RANDOM FOREST ON P450S ##
unique_reagents = p450_train['reagent'].unique().tolist()
unique_oxidants = p450_train['oxidant'].unique().tolist()
unique_solvents = p450_train['solvent'].unique().tolist()
unique_acids = p450_train['acid'].unique().tolist()
unique_additives = p450_train['additive'].unique().tolist()
unique_p450s = p450_train['P450'].unique().tolist()

In [47]:
l = 83
p450_train = p450_train.reset_index(drop=True)
p450_test = p450_test.reset_index(drop=True)

p450_train = make_label_all_df(p450_train, l)
p450_train = make_nha(p450_train)
p450_train = rxn_encoding_all_df(p450_train, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives,
                            unique_p450s, include_p450s=True)
p450_train = make_fps(p450_train)
p450_train = rf_input(p450_train)


p450_test = make_label_all_df(p450_test, l)
p450_test = make_nha(p450_test)
p450_test = rxn_encoding_all_df(p450_test, unique_reagents, unique_oxidants, unique_solvents, unique_acids, unique_additives,
                            unique_p450s, include_p450s=True)
p450_test = make_fps(p450_test)
p450_test = rf_input(p450_test)

In [48]:
rf_p450 = RandomForestRegressor()
rf_p450.fit(p450_train['rf_input'].tolist(), p450_train['Y'].tolist())

RandomForestRegressor()

In [49]:
preds_p450 = rf_p450.predict(p450_test['rf_input'].tolist())
p_p450 = snip(preds_p450, p450_test)
true_p450 = snip(p450_test['Y'].tolist(), p450_test)
p_p450 = simplify(p_p450)
rf_score_p450 = score(true_p450, p_p450)
print(rf_score_p450)

true_p 0.06528497409326425
pred_p 0.02694300518134715
tp 11 tn 887 fp 15 fn 52
19.535036175861247
